# Document Similarity & Topic Modelling

## Part 1 - Document Similarity


In [1]:
#import required libraries 
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd
from itertools import product


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    

    
    tokene = nltk.word_tokenize(doc)
    word_tokene = nltk.pos_tag(tokens=tokene)
    synsets_list = []
    for token in word_tokene:
        synsets = wn.synsets(token[0],convert_tag(token[1]))
        if len(synsets) != 0:
            synsets_list.append(synsets[0])
    
    return synsets_list


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    
    
    sin1_list = []
    max_list = []
    for sin1 in s1: 
        for sin2 in s2:
            dist = wn.path_similarity(sin1,sin2)
            if dist:
                sin1_list.append(dist)
        if len(sin1_list) != 0:
            max_list.append(max(sin1_list))
        sin1_list.clear()
    return sum(max_list)/len(max_list)


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [2]:

doc1 = 'This is a function to test document_path_similarity.'
doc2 = 'Use this function to see if your code in doc_to_synsets and similarity_score is correct!'
similarity_score(doc_to_synsets(doc2),doc_to_synsets(doc1))

0.49603174603174605

In [3]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv(r'C:\Users\sharm.LAPTOP-118C54MT\OneDrive - York University\Coursera\Course_4\Assignment_4\paraphrases.csv')
paraphrases

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...
5,1,Their difference was over whether the court sh...,Their difference was over whether the court sh...
6,1,The only announced Republican to replace Davis...,So far the only declared major party candidate...
7,1,"Druce will face murder charges, Conte said.",Conte said Druce will be charged with murder.\n
8,0,"""It's a major victory for Maine, and it's a ma...",The Maine program could be a model for other s...
9,1,Microsoft said Friday that it is halting devel...,Microsoft will stop developing versions of its...


In [4]:
#find the pair of documents in paraphrases which has the mximum similarity score 
df = paraphrases
df['similarity'] = [document_path_similarity(doc1, doc2) for doc1, doc2 in zip(paraphrases['D1'],paraphrases['D2'])] 
df.iloc[df['similarity'].idxmax(),1],df.iloc[df['similarity'].idxmax(),2]

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n')

In [5]:
#compute the similarity score for each pair of documents in paraphrase. Now classify similarity score with
#a calssification rule that if the accurayc score is greater than 0.75 classify it as 1 else 0.
#evaluate accuracy of classifer using the sklearn accuracy_score method.
from sklearn.metrics import accuracy_score
df = paraphrases
df['predicted'] = [document_path_similarity(doc1, doc2) for doc1, doc2 in zip(paraphrases['D1'],paraphrases['D2'])]
df['predicted'] = np.where((df.predicted > 0.75),1,0)
accuracy_score(df['Quality'],df['predicted'])# Your Answer Here

0.8

## Part 2 - Topic Modelling



In [6]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open(r'C:\Users\sharm.LAPTOP-118C54MT\OneDrive - York University\Coursera\Course_4\Assignment_4\newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [7]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id_map, num_topics=10, passes=25, \
                                           random_state=34)
#list of the 10 topics and the most significant 10 words in each topic.
ldamodel.print_topics(num_topics=10, num_words=10)

[(0,
  '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'),
 (1,
  '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'),
 (2,
  '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'),
 (3,
  '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'),
 (4,
  '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'),
 (5,
  '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'),
 (6,
  '0.0

In [8]:
#for the new doc find the topic distribution
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [9]:
#vectorize the document and transform it. also use sparse2coupus to convert the sparse matrix to gensim corpus
new_doc_trans = vect.transform(new_doc)

# Convert sparse matrix to gensim corpus.
new_doc_corpus = gensim.matutils.Sparse2Corpus(new_doc_trans, documents_columns=False)

#get the topic distribution 
topics = ldamodel.get_document_topics(new_doc_corpus, minimum_probability=0.01)
list(topics)[0]

[(0, 0.020003108),
 (1, 0.020003324),
 (2, 0.020001281),
 (3, 0.4967489),
 (4, 0.020004038),
 (5, 0.020004129),
 (6, 0.020002972),
 (7, 0.020002645),
 (8, 0.020003129),
 (9, 0.3432265)]